In [0]:
#service principal credentials
client_id = dbutils.secrets.get(scope="secret-scope", key="client-id")
client_secret = dbutils.secrets.get(scope="secret-scope", key="client-secret")
tenant_id = dbutils.secrets.get(scope="secret-scope", key="tenant-id")

In [0]:
#configuring OAuth authentication for Spark
storage_account = "salesdlprod1"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
storage_account="salesdlprod1"
container_name="bronze"
input_path=f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/SalesLT/Address/"
df=spark.read.format('parquet').load(input_path)
display(df)

In [0]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import TimestampType 

df = df.withColumn(
    "ModifiedDate",
    to_date(# Cast to timestamp then extract date only
        col("ModifiedDate").cast(TimestampType())
    )
)
display(df)

## Performing transformations on all tables

In [0]:
#Path to ADLS container
base_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/SalesLT/"

# Folder paths at that container
paths = dbutils.fs.ls(base_path) #baseinfo objects
# Collect table names
table_list = [item.name.rstrip('/') for item in paths] 
print(table_list)

In [0]:
for i in table_list:
    path = f"abfss://bronze@{storage_account}.dfs.core.windows.net/SalesLT/{i}/{i}.parquet"
    
    df = spark.read.format('parquet').load(path)
    
    for column_name in df.columns:
        if "date" in column_name or "Date" in column_name:
            df = df.withColumn(
                column_name,
                to_date(col(column_name).cast(TimestampType()))
            )

    output_path = f"abfss://silver@{storage_account}.dfs.core.windows.net/SalesLT/{i}/"
    
    df.write.format('delta').mode("overwrite").save(output_path)


In [0]:
display(df)